# 빅프9조

분석방법
순찰 영역은 원형이지만 격자점 내부에서는 분석을 정사각형으로 함

# 기본 설정

 필수 라이브러리 import

In [1]:
import numpy as np
import pandas as pd

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import folium
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, HTML

파일경로, csv인코딩 설정

In [2]:
data_path = "전처리완료데이터/"
raw_path = "전처리필요데이터/"
encoding = "cp949"
extension = ".csv"

경찰서 목록

In [3]:
police = np.array([37.679699, 127.055066], dtype=np.float64)

격자점 생성에 필요한 상수 선언

In [4]:
NS_DIFF_1M = 0.000007
EW_DIFF_1M = 0.000009
PATROL_HR = 900
GRID_DIFF = 90

### 데이터를 순찰 반경 안의 것만 잘라내는 함수

In [5]:
def filtering(data, police, mode = "시설"):
    distance = PATROL_HR + GRID_DIFF/2 + 5 if mode == "시설" else PATROL_HR + 10 
    ft = (data[["위도", "경도"]] - police).copy()
    ft.loc[:, "위도"] /= NS_DIFF_1M
    ft.loc[:, "경도"] /= EW_DIFF_1M
    ft = np.sqrt( np.sum( np.square(ft), axis=1 ))
    return data.loc[ft <= distance, :].reset_index(drop=True)

# 기본 격자점 생성

기본 격자점 반환함수

In [6]:
def grids(police):
    hcount = PATROL_HR//GRID_DIFF
    ew = np.arange(-hcount, hcount + 1, dtype=np.float64)
    ns = ew.reshape(-1,1).copy()
    ew *= EW_DIFF_1M * GRID_DIFF
    ns *= NS_DIFF_1M * GRID_DIFF
    
    result = np.full((2*hcount+1, 2*hcount+1, 2), police, dtype=np.float64)
    result[:,:,0] += ns
    result[:,:,1] += ew
    
    return pd.DataFrame(result.reshape(-1,2), columns=["위도", "경도"])

원형 격자점 생성

In [7]:
police_grid = grids(police)
police_grid = filtering(police_grid, police, "격자")
police_grid

,위도,경도
0,37.673399,127.054256
1,37.673399,127.055066
2,37.673399,127.055876
3,37.674029,127.051826
4,37.674029,127.052636
...,...,...
320,37.685369,127.057496
321,37.685369,127.058306
322,37.685999,127.054256
323,37.685999,127.055066


# 장소별 범죄 발생 확률 이용

### 샘플링

뽑을 개수 (5개)

In [8]:
SAMPLING = 5

격자점 범위 내에서 무작위로 5개 점 추가

In [9]:
def sampled_grids(grid):
    interval_size = SAMPLING+1
    result = np.zeros((grid.shape[0]*interval_size, 2), dtype=np.float64)
    for i in range(0, result.shape[0], interval_size):
        ran = np.zeros((interval_size, 2), dtype=np.float64)
        ran[1:] = np.random.uniform(-1, 1, (interval_size-1, 2))
        ran[:, 0] *= (GRID_DIFF/2)*NS_DIFF_1M
        ran[:, 1] *= (GRID_DIFF/2)*EW_DIFF_1M
        ran += grid.values[i//interval_size]
        result[i:i+interval_size] += ran
    return pd.DataFrame(result, columns = ["위도", "경도"])

추가한 데이터 csv내보내기

In [10]:
police_sampled = sampled_grids(police_grid)
police_sampled

,위도,경도
0,37.673399,127.054256
1,37.673120,127.054566
2,37.673431,127.054503
3,37.673477,127.054188
4,37.673255,127.054128
...,...,...
1945,37.686061,127.055476
1946,37.686120,127.055618
1947,37.685690,127.056009
1948,37.686032,127.056110


In [11]:
#police_sampled.to_csv(raw_path + "sampled_grid" + extension, index=False, encoding=encoding)

### 주소로 변환

https://www.geoservice.co.kr/

내보낸 csv 파일 이용, 위 링크에서 역지오코딩, 주소얻기

주소 기본 데이터 생성

In [12]:
def addr_data(file_path):
    data=pd.read_csv(file_path)
    data["이름"] = None
    data["확률"] = None
    data["index"]=[i//(SAMPLING+1) for i in range(data.shape[0])]
    return data

In [13]:
police_addr = addr_data(data_path + "sampled_addr.csv")
police_addr

,위도,경도,지번주소,도로명주소,이름,확률,index
0,37.673399,127.054256,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
1,37.673226,127.053969,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
2,37.673140,127.054040,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
3,37.673158,127.053899,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
4,37.673174,127.054229,서울특별시 노원구 상계동 1044,서울특별시 노원구 동일로231길 24,None,None,0
...,...,...,...,...,...,...,...
1945,37.686223,127.055503,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324
1946,37.685906,127.055661,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324
1947,37.685822,127.055937,서울특별시 노원구 상계동 산 12-1,서울특별시 노원구 동일로 1772,None,None,324
1948,37.686147,127.055588,서울특별시 노원구 상계동 1202-1,서울특별시 노원구 동일로 1772,None,None,324


# 건물 이름 크롤링

In [18]:
DELAY_TIME = 2

class kakao_map():
    def __init__(self, mode = "no"):
        self.mode = mode
        self.url = "https://map.kakao.com/"
        
    def __openweb(self):
        if self.mode == "no":
            options = wb.ChromeOptions()
            options.add_argument('headless')
            self.browser = wb.Chrome(options=options)
        else:
            self.browser = wb.Chrome()
        self.browser.get(self.url)
        time.sleep(DELAY_TIME)
    
    def __enter__(self):
        self.__openweb()
        return self
    
    def __inputclear(self):
        search_box = self.browser.find_element(By.ID, "search.keyword.query")
        while search_box.get_attribute("value"):
            search_box.clear()
     
    def __search(self, address):
        search_box = self.browser.find_element(By.ID, "search.keyword.query")
        self.__inputclear()
        search_box.send_keys(address)
        search_box.send_keys(Keys.RETURN)
        time.sleep(0.1)
    
    def find_name(self, addr, road_addr):
        if "산" in addr:
            return "산"
        if "지하" in road_addr:
            return "지하철"
        self.__search(addr)
        try:
            postage = self.browser.find_element(By.CLASS_NAME, "zip")
        except NoSuchElementException:
            return "nozip"
        else:
            postnum = postage.text
            self.__search(road_addr)
            try:
                buildname = self.browser.find_element(By.CLASS_NAME, "building")
            except NoSuchElementException:
                return postnum
            else:
                return buildname.text
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.browser.quit()
        

def search_to_csv(data, file_path, mode="no"):
    with kakao_map(mode) as browser:
        for i in range(len(data.index)):
            print(file_path[16:16+4], i)
            data.iloc[i, 4] = browser.find_name(data.iloc[i, 2], data.iloc[i, 3])
    data.to_csv(file_path, index=False, encoding=encoding)

In [ ]:
search_to_csv(police_addr, data_path+"sangge_1_sampled_name.csv", "ye")

1_sa 0
1_sa 1
1_sa 2
1_sa 3
1_sa 4
1_sa 5
1_sa 6
1_sa 7
1_sa 8
1_sa 9
1_sa 10
1_sa 11
1_sa 12
1_sa 13
1_sa 14
1_sa 15
1_sa 16
1_sa 17
1_sa 18
1_sa 19
1_sa 20
1_sa 21
1_sa 22
1_sa 23
1_sa 24
1_sa 25
1_sa 26
1_sa 27
1_sa 28
1_sa 29
1_sa 30
1_sa 31
1_sa 32
1_sa 33
1_sa 34
1_sa 35
1_sa 36
1_sa 37
1_sa 38
1_sa 39
1_sa 40
1_sa 41
1_sa 42
1_sa 43


### 이름을 토대로 유형 분류, 확률 점수 부여

In [ ]:
police_per = pd.read_csv(data_path + "sampled_name.csv", encoding=encoding)
set(police_per["이름"])

장소 분류에 따른 범죄확률

In [ ]:
crime_per = pd.read_csv(data_path+"processed_crime.csv", encoding=encoding)
crime_per

분류 결과를 바탕으로 이름이 어떤유형인지 분류

In [ ]:
def namespace(file_path):
    space = dict()
    with open(file_path, 'r', encoding='UTF8') as file:
        for i in file:
            temp=i.split(" - ")
            space[temp[0]] = temp[1].strip()
    return space

자료에 맞춰 확률 계산후 대입

In [ ]:
def insert_percentage(file_path, addr_data):
    space = namespace(file_path)
    for i in space:
        addr_data.loc[addr_data["이름"]==i, "확률"] = crime_per.loc[0, space[i]]

In [ ]:
insert_percentage(data_path + "상계1_name_category.txt", police_per)       
police_per

동일 지역에서 샘플된 데이터끼리 확률 합 구하고 격자점 데이터에 새로운 열로 추가

In [ ]:
def insert_location(grid, percentage_data):
    score = percentage_data[["index", "확률"]].groupby("index").mean()
    grid["location"] = score

In [ ]:
insert_location(police_grid, police_per)
police_grid

# 시설물 데이터 이용

시설물 점수 기준

시설물 밀도 계산 => 격자점 반경 내 개수 / 전체 범위 내 개수

이때 데이터를 순찰반경에 맞게 필터링해 쓸 것이므로 전체 범위 개수는 필터링된 데이터프레임의 행 개수이다

긍정적 시설(cctv 등)일 경우 밀도 뺌

부정적 시설(유흥업소 등)일 경우 밀도 더함

### 데이터 준비

안심시설물, cctv, 보안등 유흥업소

전체데이터는 너무 많으므로 filtering() 이용

긍정요소, 부정요소 정의

In [ ]:
failcity_data = {"ansim" : pd.read_csv(data_path+"processed_ansim.csv", encoding=encoding),
                 "cctv"  : pd.read_csv(data_path+"processed_cctv.csv", encoding=encoding),
                 "light" : pd.read_csv(data_path+"processed_light.csv", encoding=encoding),
                 "salon" : pd.read_csv(data_path+"processed_salon.csv", encoding=encoding),}
for k, v in failcity_data.items():
    failcity_data[k]=filtering(v,police)
    
positive = ["ansim", "cctv", "light"]
negative = ["salon"]

해당 범위 안에 목표물이 몇개 있는가?

In [ ]:
def location_count(data, location):
    ns_min = location.iloc[0] - GRID_DIFF/2*NS_DIFF_1M
    ns_max = location.iloc[0] + GRID_DIFF/2*NS_DIFF_1M
    ew_min = location.iloc[1] - GRID_DIFF/2*EW_DIFF_1M
    ew_max = location.iloc[1] + GRID_DIFF/2*EW_DIFF_1M
    return len(data.loc[(data["위도"] >= ns_min)&
                         (data["위도"] <= ns_max)&
                         (data["경도"] >= ew_min)&
                         (data["경도"] <= ew_max) , :].index)

격자점 데이터에 시설 밀도 열 추가하는 함수

In [ ]:
def insert_facility(grid, data):
    for k, v in data.items():
        for i in grid.index:
            grid.loc[i, k] = location_count(v, grid.loc[i, ["위도", "경도"]])
        total = grid.loc[:, k].sum()
        if total != 0:
            grid.loc[:, k] /= grid.loc[:, k].sum()
        else:
            grid.loc[:, k] = 0

밀도를 토대로 점수 계산

In [ ]:
def score(grid):
    grid["score"] = grid["location"]
    grid.drop(columns=["location"], inplace=True)
    for k in failcity_data.keys():
        mode = -1 if k in positive else 1
        grid["score"] += mode*grid[k]
        grid.drop(columns=[k], inplace=True)

밀도 계산후 열로 추가

In [ ]:
insert_facility(police_grid, failcity_data)
police_grid

점수 계산후 위경도, 점수 열만 남기기

In [ ]:
score(police_grid)
police_grid

# 지도에 시각화

지도에 해당 위도경도에 위치한 점 찍기

In [ ]:
def draw_marker(m, grid, color):
    color = "#" + color
    for i in grid[["위도","경도"]].values:
        folium.Circle(location=i, radius=2, color=color, fill_color=color).add_to(m)

시각화를 위해 점수를 -255(초록)~255(빨강)으로 변환 (min-max 스케일링 이용)

변환후 rgb코드에 맞게 문자열 생성 안전->초록, 위험->빨강

In [ ]:
def colorstr(num):
    b = num>=0
    num=hex(int(255-abs(num)))[2:]
    if len(num)==1:
        num="0"+num
    return "#ff"+num*2 if b else "#"+num+"ff"+num

In [ ]:
def score_to_color(grid):
    score = grid["score"].copy()
    scaler = MinMaxScaler(feature_range=(-255, 255))
    score = scaler.fit_transform(score.to_frame())
    return pd.Series(score.flatten())

격자점 점수 시각화

In [ ]:
def draw_score(m, grid, color):
    for i in grid.index:
        folium.Circle(location=grid.iloc[i,:2].values,
                      radius=GRID_DIFF/2,
                      color=color[i],
                      fill=True,
                      fill_color=color[i]).add_to(m)

지도객체 생성, 이름 붙이기

In [ ]:
m = folium.Map(location = police, zoom_start=60)

name="상계1"
title_html = f'<h3 align="center" style="font-size:20px"><b>{name}</b></h3>'
             
m.get_root().html.add_child(folium.Element(title_html))

경찰서, 격자점, 모든 시설물 표시

In [ ]:
folium.Marker(location=police, 
              popup='지구대',
              icon=folium.Icon(color='lightblue', icon='star')
).add_to(m)

failcity_data_color = {"ansim" : '0000ff',
                       "cctv"  : '00ffff',
                       "light" : 'ffff00',
                       "salon" : 'ff00ff'}

draw_marker(m, police_grid, '000000')
for k, v in failcity_data.items():
    draw_marker(m, v, failcity_data_color[k])

점수에서 색 문자열 생성

In [ ]:
color = score_to_color(police_grid)
color

점수 시각화

In [ ]:
draw_score(m, police_grid, color)
display(m)